In [4]:
import json

with open("/media/icnlab/Data/Manh/tinyML/FieldPlant-11/train/_annotations.coco.json", "r") as f:
    data = json.load(f)

# In các khóa chính
print("file COCO:", list(data.keys()))

file COCO: ['info', 'licenses', 'categories', 'images', 'annotations']


In [4]:
data['categories'][22]

{'id': 22, 'name': 'Tomato Brown Spots', 'supercategory': 'leaves'}

In [5]:
data['annotations'][0]

{'id': 0,
 'image_id': 0,
 'category_id': 22,
 'bbox': [1244, 742, 1409.84, 2932.98],
 'area': 4135032.523,
 'segmentation': [],
 'iscrowd': 0}

In [5]:
# cut bbox 
import os
import json
from PIL import Image

# Đường dẫn
json_path = "/media/icnlab/Data/Manh/tinyML/FieldPlant-11/train/_annotations.coco.json"
image_dir = "/media/icnlab/Data/Manh/tinyML/FieldPlant-11/train"
output_dir = "/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped"

# Tạo thư mục lưu ảnh cắt
os.makedirs(output_dir, exist_ok=True)

# Tạo dict ánh xạ từ image_id => image_info ( thông tin ảnh )
image_dict = {img_info["id"]: img_info for img_info in data["images"]}

# Tạo dict ánh xạ category_id -> category_name
category_dict = {cat["id"]: cat["name"] for cat in data["categories"]}

In [ ]:
category_dict[871]

In [35]:
img_info = image_dict[22]
img_info

{'id': 22,
 'license': 1,
 'file_name': 'Tomate_Taches_Brunes-352-_jpg.rf.014788e85715c7bb87a51addbb58ce1b.jpg',
 'height': 4160,
 'width': 3120,
 'date_captured': '2023-07-05T19:01:30+00:00'}

In [6]:
import os
%store category_dict

# image_paths = [os.path.join(image_dir, img_info["file_name"]) for img_info in image_dict.values()]
image_paths = [os.path.join(output_dir, image_name) for image_name in os.listdir('/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped')]
labels = [image_path.split('/')[-1].split('_')[-1].split('.')[0] for image_path in image_paths] 

# filter tomato class from image_paths and labels
for i in range(len(labels) - 1, -1, -1):
    if 'tomato' not in labels[i].lower():
        image_paths.pop(i)
        labels.pop(i)

%store image_paths labels

image_paths[:2], labels[:2], len(image_paths), len(labels)

Stored 'category_dict' (dict)
Stored 'image_paths' (list)
Stored 'labels' (list)


(['/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Tomate_Taches_Brunes-122-_jpg.rf.a3f5716e27e5be00a300906d166797ed_7610_Tomato Brown Spots.jpg',
  '/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Tomate_Taches_Brunes-342-_jpg.rf.14d6bc4f4f81367b610c2e671e6a8c47_704_Tomato Brown Spots.jpg'],
 ['Tomato Brown Spots', 'Tomato Brown Spots'],
 2786,
 2786)

In [ ]:
# Cắt ảnh
for ann in data["annotations"]:
    image_id = ann["image_id"]
    img_info = image_dict[image_id]

    category_id = ann["category_id"]
    category_name = category_dict[category_id]

    bbox = ann["bbox"]  # [x, y, width, height]
    img_path = os.path.join(image_dir, img_info["file_name"])
    
    # Mở ảnh
    try:
        with Image.open(img_path) as im:
            x, y, w, h = map(int, bbox)
            cropped = im.crop((x, y, x + w, y + h))

            # Tạo tên file output
            base_name = os.path.splitext(img_info["file_name"])[0]
            out_name = f"{base_name}_{ann['id']}_{category_name}.jpg"
            out_path = os.path.join(output_dir, out_name)

            # Lưu ảnh cắt
            cropped.save(out_path)
    except Exception as e:
        print(f"Lỗi với ảnh {img_path}: {e}")